In [2]:
%load_ext autoreload

In [3]:
import sys
sys.path.append("/home/aakash/College/GSoC/neural-extraction-framework/GSoC23")

In [16]:
%autoreload
from NER import ner

In [6]:
import pickle
with open("./basic_eval_data/asbtracts_and_entities.pkl","rb") as f:
    abstracts_and_entities = pickle.load(f)

In [7]:
abstracts_and_entities

{'Lionel_Messi': {'abstract': "Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi]; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Ligue 1 club Paris Saint-Germain and captains the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record seven Ballon d'Or awards, a record six European Golden Shoes, and in 2020 was named to the Ballon d'Or Dream Team. Until leaving the club in 2021, he had spent his entire professional career with Barcelona, where he won a club-record 35 trophies, including 10 La Liga titles, seven Copa del Rey titles and four UEFA Champions Leagues. A prolific goalscorer and creative playmaker, Messi holds the records for most goals in La Liga (474), most goals in a La Liga and European league season (50), most hat-tricks in La Liga (36) and the UEFA Champions League (8), and most assists in La Liga (192), a La Liga season (21) and the Cop

In [8]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [17]:
import time
for k in abstracts_and_entities:
    if len(abstracts_and_entities[k]['entities'])==0:
        l = len(abstracts_and_entities[k]['abstract'].split())
        s = time.time()
        abstracts_and_entities[k]['entities'] = ner.spacy_ner(nlp, abstracts_and_entities[k]['abstract'])
        e = time.time()
        print(f"Time taken to perform NER on abstract of {l} tokens - {e-s} seconds")

Time taken to perform NER on abstract of 734 tokens - 2.1659674644470215 seconds
Time taken to perform NER on abstract of 556 tokens - 2.3046836853027344 seconds
Time taken to perform NER on abstract of 298 tokens - 0.83363938331604 seconds
Time taken to perform NER on abstract of 403 tokens - 1.098207712173462 seconds


In [18]:
abstracts_and_entities['Lionel_Messi']

{'abstract': "Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi]; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Ligue 1 club Paris Saint-Germain and captains the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record seven Ballon d'Or awards, a record six European Golden Shoes, and in 2020 was named to the Ballon d'Or Dream Team. Until leaving the club in 2021, he had spent his entire professional career with Barcelona, where he won a club-record 35 trophies, including 10 La Liga titles, seven Copa del Rey titles and four UEFA Champions Leagues. A prolific goalscorer and creative playmaker, Messi holds the records for most goals in La Liga (474), most goals in a La Liga and European league season (50), most hat-tricks in La Liga (36) and the UEFA Champions League (8), and most assists in La Liga (192), a La Liga season (21) and the Copa América (17). H

In [24]:
# See how many entities we have in total
import pandas as pd
entities = pd.DataFrame()
for k in abstracts_and_entities:
    entities = pd.concat([entities, pd.DataFrame(abstracts_and_entities[k]['entities'])])

In [25]:
entities

,entity_group,word,start,end
0,PERSON,Lionel Andrés Messi,0,19
1,NORP,Spanh,21,28
2,DATE,24 June 1987,74,86
3,PERSON,Leo Messi,103,112
4,NORP,Argente,120,129
...,...,...,...,...
21,ORG,Project Drawdown's,1510,1528
22,CARDINAL,100,1529,1532
23,CARDINAL,around one billion,2503,2521
24,GPE,Cha,2592,2597


In [51]:
entities.to_csv("../EntityLinking/entities_from_abstracts.csv")

In [29]:
entities['entity_group'].value_counts()

entity_group
DATE           62
GPE            40
CARDINAL       34
ORG            33
PERSON         23
ORDINAL        23
EVENT          20
NORP           15
WORK_OF_ART    15
LOC            10
MONEY           3
FAC             2
QUANTITY        2
PRODUCT         2
PERCENT         1
LANGUAGE        1
Name: count, dtype: int64

In [49]:
# I won't be considering DATE and CARDINAL, ORDINAL, QUANTITY, PERCENT will look at them separately
selected_entities = entities[~entities['entity_group'].isin(
    ['CARDINAL', 'ORDINAL', 'QUANTITY', 'PERCENT', 'DATE'])]

In [50]:
selected_entities

,entity_group,word,start,end
0,PERSON,Lionel Andrés Messi,0,19
1,NORP,Spanh,21,28
3,PERSON,Leo Messi,103,112
4,NORP,Argente,120,129
5,ORG,Ligue 1,181,188
...,...,...,...,...
15,LOC,Europe,602,608
16,EVENT,World War II,691,703
21,ORG,Project Drawdown's,1510,1528
24,GPE,Cha,2592,2597
